<a href="https://colab.research.google.com/github/MaitreePimple/Data-Analytic/blob/main/Twitter_Sentiment_Data_Collection_%26_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip -q install tweepy pandas python-dotenv

In [22]:
import os, pandas as pd, tweepy

In [23]:
API_KEY = "S0sCZujdxpsYSelgAt9fuGrAF"
API_SECRET = "qoBBvAujyjW1Nw0U5dJt4TnynfKEeaI8DRNrjTdMrvqtDBDLsv"
ACCESS_TOKEN = "1958093337820266497-xzpYNXZ0fuhrAimiHmzHKqVgFvn11i"
ACCESS_SECRET = "N345iFbZltmFnPWLDpgADBnKigl2txtXYkLmmmRS3P7jG"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABCq3gEAAAAAKbFG2vW%2BdNtinr3HvhCBJrXpvAQ%3DkAQ0fdaTIDRtDhai8fnaz2IL77n6vsiDuDZXBVKxHm5RMYm2Uv"

In [24]:
client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    consumer_key=API_KEY,
    consumer_secret=API_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_SECRET,
    wait_on_rate_limit=True
)

In [25]:
try:
    test = client.get_user(username="TwitterDev")
    print("✅ Auth OK. Example API call status:", "OK" if test.data else "No data (but auth fine)")
except Exception as e:
    print("❌ Auth/API error:", e)

✅ Auth OK. Example API call status: No data (but auth fine)


In [35]:
TOPIC = 'Data Collection and Cleaning'
LANG  = "en"
LIMIT = 1000

query = f'"{TOPIC}" lang:{LANG} -is:retweet -is:reply'

tweet_fields = ["created_at", "public_metrics", "lang", "author_id"]
user_fields  = ["username", "name"]
expansions   = ["author_id"]

rows = []
collected = 0

for resp in tweepy.Paginator(
        client.search_recent_tweets,
        query=query,
        tweet_fields=tweet_fields,
        user_fields=user_fields,
        expansions=expansions,
        max_results=100  # per page (API max)
    ):
    if not resp.data:
        continue


    users = {u["id"]: u for u in (resp.includes.get("users", []) if resp.includes else [])}

    for t in resp.data:
        u = users.get(t.author_id, {})
        pm = t.public_metrics or {}

        rows.append({
            "id": t.id,
            "date": t.created_at.isoformat() if t.created_at else None,
            "username": u.get("username"),
            "displayname": u.get("name"),
            "raw_text": t.text,
            "likeCount": pm.get("like_count"),
            "retweetCount": pm.get("retweet_count"),
            "replyCount": pm.get("reply_count"),
            "quoteCount": pm.get("quote_count"),
            "url": f'https://twitter.com/{u.get("username","")}/status/{t.id}'
        })

    collected += len(resp.data)
    if collected >= LIMIT:
        break

print(f"Collected {len(rows)} tweets")
df = pd.DataFrame(rows)
df.head()

Collected 2 tweets


,id,date,username,displayname,raw_text,likeCount,retweetCount,replyCount,quoteCount,url
0,1956634210958979147,2025-08-16T08:28:43+00:00,SimoneHogg34687,Eureka Innovator,Data quality directly impacts AI performance. ...,0,0,0,0,https://twitter.com/SimoneHogg34687/status/195...
1,1955929132585337015,2025-08-14T09:46:59+00:00,KingHenryMorgan,Ogbuefi Chuke,data collection and cleaning. https://t.co/vv9...,0,0,0,0,https://twitter.com/KingHenryMorgan/status/195...


In [37]:
df = df.drop_duplicates(subset=["id"]).dropna(subset=["raw_text"]).reset_index(drop=True)

out_path = "tweets_raw.csv"
df.to_csv(out_path, index=False)

print(f"✅ Saved {len(df)} rows to {out_path}")
df.sample(min(5, len(df)), random_state=42)

✅ Saved 2 rows to tweets_raw.csv


,id,date,username,displayname,raw_text,likeCount,retweetCount,replyCount,quoteCount,url
1,1955929132585337015,2025-08-14T09:46:59+00:00,KingHenryMorgan,Ogbuefi Chuke,data collection and cleaning. https://t.co/vv9...,0,0,0,0,https://twitter.com/KingHenryMorgan/status/195...
0,1956634210958979147,2025-08-16T08:28:43+00:00,SimoneHogg34687,Eureka Innovator,Data quality directly impacts AI performance. ...,0,0,0,0,https://twitter.com/SimoneHogg34687/status/195...


In [42]:
import pandas as pd
df = pd.read_csv("tweets_raw.csv")

print("Shape of dataset:", df.shape)
print("\nMissing values:\n", df.isnull().sum())
print("\nDuplicate tweets:", df.duplicated(subset=['raw_text']).sum())

print("\nSummary statistics for likes/retweets:")
print(df[['likeCount', 'retweetCount']].describe())

Shape of dataset: (2, 10)

Missing values:
 id              0
date            0
username        0
displayname     0
raw_text        0
likeCount       0
retweetCount    0
replyCount      0
quoteCount      0
url             0
dtype: int64

Duplicate tweets: 0

Summary statistics for likes/retweets:
       likeCount  retweetCount
count        2.0           2.0
mean         0.0           0.0
std          0.0           0.0
min          0.0           0.0
25%          0.0           0.0
50%          0.0           0.0
75%          0.0           0.0
max          0.0           0.0


In [44]:
import re

def clean_tweet(text):
    text = re.sub(r"http\S+", "", text)         # Remove URLs
    text = re.sub(r"@\w+", "", text)           # Remove mentions
    text = re.sub(r"#", "", text)              # Remove hashtag symbol
    text = re.sub(r"\s+", " ", text).strip()   # Remove extra spaces
    return text

df['clean_tweet'] = df['raw_text'].apply(clean_tweet)

# Save cleaned data
df.to_csv("tweets_cleaned.csv", index=False)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['clean_tweet'])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.to_csv("tweets_tfidf.csv", index=False)
